In [1]:
import numpy as np
from keras.layers import *
from keras.models import Model


/home/anchen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def conv_block(x, filters, size, stride=(2,2), mode='same', act=True):
    x = Convolution2D(filters, (size, size), strides=stride, padding=mode)(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x) if act else x

In [3]:
def res_block(ip, nf=64):
    x = conv_block(ip, nf, 3, (1,1))
    x = conv_block(x, nf, 3, (1,1), act=False)
    return Add()([x, ip])

In [4]:
def up_block(x, filters, size):
    x = UpSampling2D()(x)
    x = Convolution2D(filters, (size, size), padding='same')(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x)

In [5]:
def get_model():
    inp=Input((None, None, 3))
    x=conv_block(inp, 64, 9, (1,1))
    for i in range(4): x=res_block(x)
    x=up_block(x, 64, 4)
    x=up_block(x, 64, 4)
    x=Conv2D(3, (9, 9), activation="tanh", padding="same")(x)
    outp=Lambda(lambda x: (x+1)*127.5)(x)
    return Model(inp,outp)

In [6]:
model = get_model()

In [7]:
model.load_weights('super-resolution.h5')

In [ ]:
from PIL import Image

img = Image.open("test.jpg")
pred_img = np.expand_dims(np.array(img), axis=0)
new_p = model.predict(pred_img)

In [ ]:
Image.fromarray(new_p[0].astype('uint8'))